# **This is a database of Online Sport Clothing Company**
The database consists of 5 different tables:
- info.csv
- finance.csv
- reviews.csv
- traffic.csv
- brands.csv

In [5]:
#importing packages
import pandas as pd
import numpy as np

# Read in the data
info = pd.read_csv("info.csv")
finance = pd.read_csv("finance.csv")
reviews = pd.read_csv("reviews.csv")
traffic = pd.read_csv("traffic.csv")
brands = pd.read_csv("brands.csv")

# Merge the data
merged_df = info.merge(finance, on="product_id", how="outer")
merged_df = merged_df.merge(reviews, on="product_id", how="outer")
merged_df = merged_df.merge(traffic, on="product_id", how="outer")
merged_df = merged_df.merge(brands, on="product_id", how="outer")

# Drop null values
merged_df.dropna(inplace=True)

merged_df.head()

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,last_visited,brand
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,2018-11-29 16:16:00,Adidas
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,2018-02-01 10:27:00,Adidas
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,2018-09-07 20:06:00,Adidas
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,2019-07-18 15:26:00,Adidas
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,2019-01-30 12:09:00,Adidas


## What is the volume of products and average revenue for Adidas and Nike products based on price quartiles?


In [4]:
# Add price labels based on listing_price quartiles
merged_df["price_label"] = pd.qcut(merged_df["listing_price"], 4, labels=["Budget", "Average", "Expensive", "Elite"])

# Group by brand and price_label to get volume and mean revenue
adidas_vs_nike = merged_df.groupby(["brand", "price_label"]).agg({"price_label": "count", "revenue": "mean"})

adidas_vs_nike

price_label      revenue
brand  price_label                          
Adidas Budget               538  2050.966580
       Average              599  2982.297429
       Expensive            707  4599.578600
       Elite                533  8424.178574
Nike   Budget               321  1664.329595
       Average                8   675.592500
       Expensive             43   472.739070
       Elite                124  1418.420484

## Do any differences exist between the word count of a product's description and its mean rating?

In [6]:
# Upper description length limits
lengthes = [0, 99, 199, 299, 399, 499, 599, 699]

# Description length labels
labels = ["99", "199", "299", "399", "499", "599", "699"]

# Store the length of each description
merged_df["word_limit"] = merged_df["description"].str.len()

# Cut into bins
merged_df["word_limit"] = pd.cut(merged_df["word_limit"], bins=lengthes, labels=labels)

# Group by the bins
descriptions = merged_df.groupby("word_limit", as_index=False).agg({"rating": "mean", "reviews": "count"})

descriptions

,word_limit,rating,reviews
0,99,1.866667,6
1,199,3.188937,461
2,299,3.287108,1660
3,399,3.313765,603
4,499,3.396460,113
5,599,3.120000,15
6,699,3.653333,15


## How does the volume of products and median revenue vary between clothing and footwear?

In [7]:
# Copy the DataFrame to avoid overwriting or filtering the original data
shoes = merged_df.copy(deep=True)

# List of footwear keywords
mylist = "shoe*|trainer*|foot*"

# Filter for footwear products
shoes = merged_df[merged_df["description"].str.contains(mylist)]

# Filter for clothing products
clothing = merged_df[~merged_df.isin(shoes["product_id"])]

# Remove null product_id values from clothing DataFrame
clothing.dropna(inplace=True)

# Create product_types DataFrame
product_types = pd.DataFrame({"clothing_products": len(clothing), 
                              "clothing_revenue": clothing["revenue"].median(), 
                              "footwear_products": len(shoes), 
                              "footwear_revenue": shoes["revenue"].median()}, 
                              index=[0])

product_types

,clothing_products,clothing_revenue,footwear_products,footwear_revenue
0,439,683.73,2434,3073.3
